In [1]:
from core import *

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

ds_baseline, ds_baseline_test = pd.read_csv('../data/feed/feed_baseline.csv'), pd.read_csv('../data/feed/feed_baseline_test.csv')
ds_baseline

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,month
0,1,0,0,1,9.84,14.395,81,0.0000,16,0,1
1,1,0,0,1,9.02,13.635,80,0.0000,40,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,32,2,1
3,1,0,0,1,9.84,14.395,75,0.0000,13,3,1
4,1,0,0,1,9.84,14.395,75,0.0000,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,336,19,12
10882,4,0,1,1,14.76,17.425,57,15.0013,241,20,12
10883,4,0,1,1,13.94,15.910,61,15.0013,168,21,12
10884,4,0,1,1,13.94,17.425,61,6.0032,129,22,12


In [3]:
from sklearn.linear_model import LinearRegression
from core import *

X_bs, y_bs = ds_baseline.drop('count', axis=1), ds_baseline['count']

baseline = LinearRegression()
cross_val_score(baseline,
                X_bs, y_bs,
                scoring=make_scorer(rmsle)).mean()

1.337817333580025

In [4]:
baseline.fit(X_bs, y_bs)
make_sub(baseline.predict(ds_baseline_test), '../data/out/bs.csv')

In [5]:
from sklearn.ensemble import RandomForestRegressor

eval_model(RandomForestRegressor(), X_bs, y_bs)

0.5864893538361275


In [33]:
eval_data(RandomForestRegressor(),
          '../data/feed/feed_2[outliers,day of week,groupby].csv',
          '../data/feed/feed_2[outliers,day of week,groupby]_test.csv',
          '../data/out/rf_2.csv',
          export_test_set=True)

In [34]:
from core import *

def create_rf_model(trial):
    return RandomForestRegressor(
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 15),
        min_samples_split=trial.suggest_uniform("min_samples_split", 0.05, 1.0),
        n_estimators=trial.suggest_int("n_estimators", 2, 300),
        max_depth=trial.suggest_int("max_depth", 2, 15),
        random_state=666
    )

optimize(create_rf_model,
         path='../data/feed/feed_2[outliers,day of week,groupby].csv',
         direction='minimize',
         trials=200)

In [35]:
optimize('RFR',
         path='../data/feed/feed_2[outliers,day of week,groupby].csv',
         direction='minimize',
         trials=50)

In [37]:
from xgboost import XGBRegressor
from mlens.ensemble import SuperLearner
from sklearn.tree import DecisionTreeRegressor

model = SuperLearner()
model.add([
    LGBMRegressor(**{'learning_rate': 0.013264625110413585,
                     'n_estimators': 406, 'max_depth': 25,
                     'num_leaves': 1253, 'min_child_samples': 10}),
    XGBRegressor(**{'learning_rate': 0.006438488538697672,
                    'n_estimators': 579, 'max_depth': 12,
                    'gamma': 0.9267817436177146})
])
model.add_meta(RandomForestRegressor(**{'min_samples_leaf': 2,
                             'min_samples_split': 0.010039195988554477,
                             'n_estimators': 231, 'max_depth': 16}))
eval_data(model,
    '../data/feed/feed_baseline.csv',
    '../data/feed/feed_baseline_test.csv',
    '../data/out/rf_2.csv',
    export_test_set=True
)

In [26]:
from xgboost import XGBRegressor
import math
from core import eval_data
from sklearn.ensemble import GradientBoostingRegressor

# xgb_model = XGBRegressor(**{'learning_rate': 0.13190716148710585, 'n_estimators': 161, 'max_depth': 6, 'gamma': 0.046580638517337444})
xgb_model = GradientBoostingRegressor(n_estimators=4000,alpha=0.01)

eval_data(xgb_model,
    '../data/feed/feed_3[log].csv',
    '../data/feed/feed_3[log]_test.csv',
    '../data/out/!gb_3[no].csv',
    evaluate=False,
    export_test_set=True,
    target_transform_fn=np.expm1
)

In [25]:
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(**{'learning_rate': 0.06325519603377378, 'n_estimators': 785, 'max_depth': 22, 'num_leaves': 1973, 'min_child_samples': 157})

eval_data(lgb_model,
    '../data/feed/feed_3[log].csv',
    '../data/feed/feed_3[log]_test.csv',
    '../data/out/!lgb_3.csv',
    evaluate=False,
    export_test_set=True,
    target_transform_fn=np.expm1
)